In [27]:
import pandas as pd 
from helpful import *

In [28]:
# Загружаем обработанные данные
df = pd.read_excel('data/data.xlsx')

In [29]:
# Для каждого значения merchant_name находим максимум и минимум cashback в виде словаря {название : [min, max]}
minimax = {}
for merchant in df['merchant_name'].unique():
    minimax[merchant] = [df[df['merchant_name'] == merchant]['cashback'].min(), df[df['merchant_name'] == merchant]['cashback'].max()]


In [30]:
# Преобразуем значения cashback в диапазон от 0 до 1 в завимости от максимума и минимума для каждого merchant_name
transformed = df.copy()
for merchant in minimax:
    transformed.loc[transformed['merchant_name'] == merchant, 'cashback'] = (transformed[transformed['merchant_name'] == merchant]['cashback'] - minimax[merchant][0]) / (minimax[merchant][1] - minimax[merchant][0])

In [31]:
transformed

,merchant_name,day,month,cashback
0,1С Интерес,2022-01-21 00:00:00,2022-01-01 00:00:00,0.071747
1,1С Интерес,2022-01-22 00:00:00,2022-01-01 00:00:00,0.148351
2,1С Интерес,2022-01-23 00:00:00,2022-01-01 00:00:00,0.141703
3,1С Интерес,2022-01-24 00:00:00,2022-01-01 00:00:00,0.056758
4,1С Интерес,2022-01-25 00:00:00,2022-01-01 00:00:00,0.130291
...,...,...,...,...
27537,Яндекс.Еда и Лавка,2023-01-27 00:00:00,2023-01-01 00:00:00,0.369006
27538,Яндекс.Еда и Лавка,2023-01-28 00:00:00,2023-01-01 00:00:00,0.464632
27539,Яндекс.Еда и Лавка,2023-01-29 00:00:00,2023-01-01 00:00:00,0.350585
27540,Яндекс.Еда и Лавка,2023-01-30 00:00:00,2023-01-01 00:00:00,0.206159
